In [1]:
import pandas as pd
import sqlite3 as s3
from sklearn.cluster import KMeans
import math
import matplotlib.pyplot as plt
import warnings
import numpy as np
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.frequent_patterns import apriori
warnings.simplefilter('ignore')

In [2]:
go_sales = s3.connect('go_sales.sqlite')
go_crm = s3.connect('go_crm.sqlite')
df = pd.read_sql_query('SELECT * FROM product', go_sales)
df = pd.merge(df, pd.read_sql_query('SELECT * FROM order_details', go_sales), on='PRODUCT_NUMBER')
df

,PRODUCT_NUMBER,INTRODUCTION_DATE,PRODUCT_TYPE_CODE,PRODUCTION_COST,MARGIN,PRODUCT_IMAGE,LANGUAGE,PRODUCT_NAME,DESCRIPTION,TRIAL888,ORDER_DETAIL_CODE,ORDER_NUMBER,QUANTITY,UNIT_COST,UNIT_PRICE,UNIT_SALE_PRICE,TRIAL879
0,1,15-2-2011,1,4,.33,P01CE1CG1.jpg,EN,TrailChef Water Bag,"Lightweight, collapsible bag to carry liquids ...",T,100085,8474,40,4.01,6.03,6.03,T
1,1,15-2-2011,1,4,.33,P01CE1CG1.jpg,EN,TrailChef Water Bag,"Lightweight, collapsible bag to carry liquids ...",T,100086,8470,40,4.01,6.03,6.03,T
2,1,15-2-2011,1,4,.33,P01CE1CG1.jpg,EN,TrailChef Water Bag,"Lightweight, collapsible bag to carry liquids ...",T,100087,8464,48,4.01,6.03,6.03,T
3,1,15-2-2011,1,4,.33,P01CE1CG1.jpg,EN,TrailChef Water Bag,"Lightweight, collapsible bag to carry liquids ...",T,100088,9254,66,4.01,6.03,6.03,T
4,1,15-2-2011,1,4,.33,P01CE1CG1.jpg,EN,TrailChef Water Bag,"Lightweight, collapsible bag to carry liquids ...",T,100089,8472,88,4.01,6.03,6.03,T
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43058,99,15-2-2011,17,2,.6,P99OP4FA17.jpg,EN,Aloe Relief,"Perfect for minor burns and sunburn, the aloe ...",T,98478,8210,30,1.97,4.94,4.94,T
43059,99,15-2-2011,17,2,.6,P99OP4FA17.jpg,EN,Aloe Relief,"Perfect for minor burns and sunburn, the aloe ...",T,99566,8246,20,2,5,3.95,T
43060,99,15-2-2011,17,2,.6,P99OP4FA17.jpg,EN,Aloe Relief,"Perfect for minor burns and sunburn, the aloe ...",T,99567,8239,18,2,5,5,T
43061,99,15-2-2011,17,2,.6,P99OP4FA17.jpg,EN,Aloe Relief,"Perfect for minor burns and sunburn, the aloe ...",T,99568,8270,26,2,5,5,T


In [3]:
df = df.loc[:, ['PRODUCT_NAME', 'ORDER_NUMBER']]
df

,PRODUCT_NAME,ORDER_NUMBER
0,TrailChef Water Bag,8474
1,TrailChef Water Bag,8470
2,TrailChef Water Bag,8464
3,TrailChef Water Bag,9254
4,TrailChef Water Bag,8472
...,...,...
43058,Aloe Relief,8210
43059,Aloe Relief,8246
43060,Aloe Relief,8239
43061,Aloe Relief,8270


In [4]:
df = df.groupby('ORDER_NUMBER')['PRODUCT_NAME'].apply(list).tolist()
df


[['Insect Bite Relief',
  'TrailChef Canteen',
  'Firefly 2',
  'Firefly Multi-light',
  'EverGlow Single',
  'EverGlow Butane',
  'BugShield Extreme'],
 ['TrailChef Water Bag', 'Canyon Mule Extreme Backpack', 'Firefly Extreme'],
 ['TrailChef Canteen',
  'EverGlow Double',
  'EverGlow Lamp',
  'Seeker 50',
  'Sun Blocker',
  'Sun Shelter Stick',
  'Sun Shield'],
 ['Firefly Multi-light',
  'TrailChef Double Flame',
  'Sun Blocker',
  'Sun Shelter 30',
  'Aloe Relief'],
 ['Hibernator Extreme',
  'TrailChef Double Flame',
  'Sun Shelter 15',
  'Sun Shelter 30',
  'Calamine Relief',
  'Aloe Relief'],
 ['Insect Bite Relief',
  'Hibernator Pad',
  'Canyon Mule Journey Backpack',
  'TrailChef Kitchen Kit',
  'Firefly Multi-light',
  'EverGlow Lamp',
  'Bear Survival Edge'],
 ['TrailChef Water Bag', 'Canyon Mule Cooler'],
 ['Star Gazer 2',
  'TrailChef Cook Set',
  'Mountain Man Combination',
  'Polar Sports',
  'BugShield Lotion Lite'],
 ['TrailChef Water Bag',
  'Star Gazer 2',
  'Hibernator

In [5]:
te = TransactionEncoder()
te_ary = te.fit(df).transform(df)
one_hot_df = pd.DataFrame(te_ary, columns=te.columns_)
one_hot_df

,Aloe Relief,Bear Edge,Bear Survival Edge,Blue Steel Max Putter,Blue Steel Putter,BugShield Extreme,BugShield Lotion,BugShield Lotion Lite,BugShield Natural,BugShield Spray,...,TrailChef Canteen,TrailChef Cook Set,TrailChef Cup,TrailChef Deluxe Cook Set,TrailChef Double Flame,TrailChef Kettle,TrailChef Kitchen Kit,TrailChef Single Flame,TrailChef Utensils,TrailChef Water Bag
0,False,False,False,False,False,True,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
2,False,False,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
3,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
4,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5355,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5356,False,False,False,False,False,True,False,False,False,False,...,False,True,True,False,False,False,False,True,True,False
5357,True,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,True,False,False,False
5358,False,False,False,False,False,False,False,False,False,False,...,True,False,False,False,True,False,False,False,True,True


In [6]:
frequent_itemsets = apriori(one_hot_df, min_support=0.025, use_colnames=True)
frequent_itemsets

#min_support betekend dat het product minimaal 2.5% van de tijd voorkomt in de dataset

,support,itemsets
0,0.057463,(Aloe Relief)
1,0.082649,(Bear Edge)
2,0.073507,(Bear Survival Edge)
3,0.066791,(Blue Steel Max Putter)
4,0.072761,(Blue Steel Putter)
...,...,...
115,0.025373,"(Granite Axe, Firefly Climbing Lamp)"
116,0.025933,"(Granite Extreme, Firefly Climbing Lamp)"
117,0.029104,"(Granite Axe, Granite Extreme)"
118,0.025746,"(Granite Hammer, Granite Grip)"


In [7]:
rules = association_rules(frequent_itemsets, metric='confidence', min_threshold=0.025)
rules

# min_threshold betekend hoevaak de waarheid wordt verteld

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(Hailstorm Titanium Irons),(Blue Steel Max Putter),0.077985,0.066791,0.025746,0.330144,4.942931,0.020538,1.393148,0.865160
1,(Blue Steel Max Putter),(Hailstorm Titanium Irons),0.066791,0.077985,0.025746,0.385475,4.942931,0.020538,1.500370,0.854783
2,(Granite Axe),(Firefly Climbing Lamp),0.084701,0.073694,0.025373,0.299559,4.064908,0.019131,1.322462,0.823766
3,(Firefly Climbing Lamp),(Granite Axe),0.073694,0.084701,0.025373,0.344304,4.064908,0.019131,1.395919,0.813977
4,(Granite Extreme),(Firefly Climbing Lamp),0.086567,0.073694,0.025933,0.299569,4.065037,0.019553,1.322480,0.825457
5,(Firefly Climbing Lamp),(Granite Extreme),0.073694,0.086567,0.025933,0.351899,4.065037,0.019553,1.409398,0.813986
6,(Granite Axe),(Granite Extreme),0.084701,0.086567,0.029104,0.343612,3.969315,0.021772,1.391606,0.817293
7,(Granite Extreme),(Granite Axe),0.086567,0.084701,0.029104,0.336207,3.969315,0.021772,1.378891,0.818963
8,(Granite Hammer),(Granite Grip),0.071642,0.079104,0.025746,0.359375,4.543042,0.020079,1.437495,0.840067
9,(Granite Grip),(Granite Hammer),0.079104,0.071642,0.025746,0.325472,4.543042,0.020079,1.376307,0.846875
